#Trabalho Final NLP
Wesley Pedro Scorsatto e Mateus Girardi

In [1]:
#Importando as bibliotecas necessárias para o projeto
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

<ipython-input-1-d42b033a4446>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
path = "roteiros.csv"
df = pd.read_csv(path)

In [6]:
df.head()

,Unnamed: 0,file_name,text_raw,file_name_short,movie_name
0,0,12.0.812.891.pdf,"Untitled-1 1 4/12/2009, 18:56 O Caçador de...",12.0.812.891,"CAÇADOR DE DIAMANTES, O"
1,1,12.0.812.942.pdf,"Untitled-1 1 10/12/2009, 22:40 Dois Córre...",12.0.812.942,DOIS CÓRREGOS
2,2,12.0.812.943.pdf,"Untitled-10 1 7/12/2009, 11:39 O Caso dos ...",12.0.812.943,"CASO DOS IRMÃOS NAVES, O"
3,3,12.0.812.978.pdf,"Untitled-4 1 7/12/2009, 18:07 Como Fazer u...",12.0.812.978,COMO FAZER UM FILME DE AMOR
4,4,12.0.813.054.pdf,"Bens Confisc capa.pmd 1 8/12/2009, 15:16 B...",12.0.813.054,BENS CONFISCADOS


In [7]:
df["text_raw"][0]

'Untitled-1  1  4/12/2009, 18:56   O Caçador de Diamantes   Governador Geraldo Alckmin  Secretário Chefe da Casa Civil  Arnaldo  Madeira  Imprensa Oficial do Estado de São Paulo  Diretor-presidente Diretor  Vice-presidente Diretor  Industrial Diretor Financeiro e Administrativo  Hubert  Alquéres Luiz Carlos Frigerio Teiji Tomioka  Flávio  Capello  Núcleo de Projetos Institucionais Projetos  Editoriais  Emerson Bento Pereira Vera Lucia Wey  Coleção Aplauso Cinema Brasil  Coordenador  Geral Coordenador  Operacional  Rubens Ewald Filho  e Pesquisa Iconográfica Marcelo  Pestana  Projeto  Gráfico Revisão e  Editoração  Carlos  Cirne         Vittorio Capellaro  O Caçador de Diamantes  por Maximo Barro  São Paulo, 2004   Ficha catalográfica elaborada pela Biblioteca da Imprensa Oficial do Estado  Capellaro,  Victor  O caçador de diamantes: o primeiro roteiro cinematográfico brasileiro completo/Victor Capellaro; analisado por Maximo Barro. – São Paulo: Impren- sa Oficial do Estado de São Paulo

In [8]:
for a in ["split","regex","NLTK"]:
    nwords = 0;
    for p in df["text_raw"]:
        nwords+=len(p.split() if a == "split" else re.findall(r"\w+(?:'\w+)?|[^\w\s]", p) if a == "regex" else nltk.word_tokenize(p, language="portuguese"));
    print(a+": "+str(nwords));

split: 1776586
regex: 2295574
NLTK: 2132079


In [9]:
ourStopWords = ["copyright","untitled"]

In [13]:
#pre-processamento dos dados
def preprocessamento(frase):
  tokens = re.findall(r"\w+(?:'\w+)?|[^\w\s]", frase)
  tokens = [w.lower() for w in tokens];
  portugues_stops = stopwords.words("portuguese");
  tokens = [w.lower() for w in tokens if w not in portugues_stops and w not in ourStopWords];
  translator = str.maketrans(string.punctuation, ' '*len(string.punctuation));
  tokens = [w.translate(translator) for w in tokens if len(w) > 1 and w != " "];
  tokens = [w for w in tokens if len(w.replace(" ","")) > 1];
  return tokens;

In [126]:
# Função distancia de cosseno - Bag of Words (BOW) 
def distance_cosseno_bag(frase, df):
    sep = ' '
    d = []

    #Pré-processamento da Frase
    frase = preprocessamento(frase)
    t1 = [sep.join(frase)]

    for i, infos in df.iterrows():
        texto = preprocessamento(df["text_raw"][i])
        t1.append(sep.join(texto))   
        vect_bag = CountVectorizer(binary=True)   
        y = vect_bag.fit_transform(t1).toarray()
        v = distance.cosine(y[0], y[i+1]) 
        d.append([v, i, df["movie_name"][i]])
    r = sorted(d)
    print(r[0:3])

In [167]:
# Mídia - RÉ BEMOL
frase1 = 'Ele desliga o telefone e volta a falar com seu amigo.'
frase2 = 'Ele parece um presente ou um cantor de ópera.'
frase3 = 'Uma barrinha que você aperta e por um momento a menina vive seu amor quando era linda.'
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.8848366400737804, 45, 'RÉ BEMOL '], [0.9119169670727945, 103, 'O PASSAGEIRO OBSCURO '], [0.9138339163735673, 81, 'INVERNO']]
[[0.8969947594750791, 45, 'RÉ BEMOL '], [0.9575140711337913, 96, 'NOVA BANDEIRA PARA A NAÇÃO'], [0.9604406113935382, 115, 'RATOEIRA']]
[[0.8637368749173335, 45, 'RÉ BEMOL '], [0.9271763678098273, 81, 'INVERNO'], [0.9381961999383319, 86, 'LIÇÕES DE QUÍMICA']]


In [168]:
# Mídia - GASOLINA COMUM
frase1 = 'Um barulho muito alto os interrompe'
frase2 = 'Regina abre o livro, olha para Carlo e começa a ler, apoiando suas costas na porta e esticando os pés sobre Carlo'
frase3 = 'Um garotão barbudo de vinte e poucos anos sai de casa sem dar bola para o carro, conformado'
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.9241901956421097, 103, 'O PASSAGEIRO OBSCURO '], [0.9315346803118543, 77, 'GASOLINA COMUM'], [0.9419415250212863, 118, 'ENSAIO']]
[[0.8574780718626078, 77, 'GASOLINA COMUM'], [0.8647327210895455, 73, 'A ESPERA'], [0.9142944593028991, 45, 'RÉ BEMOL ']]
[[0.868898893978731, 77, 'GASOLINA COMUM'], [0.9206480399141845, 64, 'PLETORA DE IRMA'], [0.9211583472014556, 91, 'MORANGO']]


In [169]:
# Mídia - NUNCA DEIXE DE ME OLHAR
frase1 = 'Por favor, me acompanhe em um martini, então.'
frase2 = 'Fernanda balança a cabeça negativamente.'
frase3 = 'Uma TV de 49 polegadas exibe um jogo de futebol.'
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.9490352808562375, 38, 'ENCOMENDA, A '], [0.950970966215454, 33, 'AKASHA'], [0.951549841688849, 30, 'ACORDA']]
[[0.9329598476846009, 118, 'ENSAIO'], [0.9343467835701387, 103, 'O PASSAGEIRO OBSCURO '], [0.9431017568641838, 63, ' LATA, A']]
[[0.9423449939468246, 97, 'NUNCA DEIXE DE ME OLHAR'], [0.9575396112195776, 117, 'DO MUNDO NADA SE LEVA'], [0.9606707435882719, 81, 'INVERNO']]


In [170]:
# Mídia - PICOLÉ BOIADO
frase1 = 'O rapaz se levanta com as pernas trêmulas à procura de ajuda.'
frase2 = 'Os dois permanecem admirando a mulher até a perderem de vista.'
frase3 = 'Uma jovem do grupo volta em direção a Cláudio.'
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.8667653224947017, 106, 'PICOLÉ BOIADO'], [0.9341573078040779, 101, 'O QUE SOBE, DESCE '], [0.9427987444186599, 105, 'PEUR ET ABANDON']]
[[0.8667653224947017, 106, 'PICOLÉ BOIADO'], [0.9410061153824079, 81, 'INVERNO'], [0.9463943732581103, 103, 'O PASSAGEIRO OBSCURO ']]
[[0.87837393614737, 106, 'PICOLÉ BOIADO'], [0.92815787918929, 50, 'CAFÉ DA TARDE'], [0.9321448856256023, 63, ' LATA, A']]


In [171]:
# Mídia - INVERNO
frase1 = 'Ana desliga o telefone e dá partida no carro.'
frase2 = 'Vai dar tudo certo, meu amor, não fica assim.'
frase3 = 'A imagem do céu é refletida em um dos olhos de Ana.'
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.8820122307648156, 81, 'INVERNO'], [0.9237507148336976, 85, 'LEO 1313'], [0.9288131503185626, 66, 'DISPAROS']]
[[0.8725586436671976, 81, 'INVERNO'], [0.8784766175197074, 82, 'QUERO SER JACK WHITE '], [0.8832030356434287, 45, 'RÉ BEMOL ']]
[[0.892292395466959, 81, 'INVERNO'], [0.911726517049525, 64, 'PLETORA DE IRMA'], [0.9122941980692971, 33, 'AKASHA']]


In [172]:
# Mídia - MEMÓRIAS PÓSTUMAS
frase1 = "Podemos talvez chegar até fatos importantes dos anos 90"
frase2 = "Não tive filhos, não transmiti a nenhuma criatura o legado da nossa miséria"
frase3 = "Um velho escravo anda pela casa vazia"
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.9258750683338899, 91, 'MORANGO'], [0.9451911928568598, 92, 'MORTE'], [0.9466330705041162, 61, 'DEUS EX-MACHINA']]
[[0.959708851798731, 89, 'MEMÓRIAS PÓSTUMAS'], [0.9727295263922994, 59, 'DESINIBIDA DO GRAJAÚ, A'], [0.9731392345324873, 64, 'PLETORA DE IRMA']]
[[0.9220187132634945, 66, 'DISPAROS'], [0.9304391656359747, 79, 'HOMENS   AO MAR'], [0.9345346329292022, 83, 'JANELA SOBRE O SONHO']]


In [173]:
# Mídia - SEXO & BEETHOVEN - O REENCONTRO
frase1 = "Infelizmente, não dividimos bucetas"
frase2 = "Nem os deuses tinham culpa"
frase3 = "Eu vou pedir ela em casamento"
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.9478241142959526, 120, 'SEXO & BEETHOVEN - O REENCONTRO'], [0.9747058254628652, 41, 'JOB THAT SLOWLY KILLS YOU, A'], [0.9785577493032441, 68, 'DONA CARMELA']]
[[0.9573985677157695, 120, 'SEXO & BEETHOVEN - O REENCONTRO'], [0.9699849887406168, 69, 'DUDA, A CARTEIRA'], [0.9737387134280555, 68, 'DONA CARMELA']]
[[0.9245148643903602, 91, 'MORANGO'], [0.9315346803118543, 77, 'GASOLINA COMUM'], [0.9471852266980154, 65, 'DIANTE DA LEI']]


In [174]:
# Mídia - VINGANÇA
frase1 = "Já não chega o que a gente está passando?"
frase2 = "Seu canalha. Eu sei de tudo!!!"
frase3 = "Eu to fazendo um puxadinho"
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.9154845745271484, 83, 'JANELA SOBRE O SONHO'], [0.9162781641721078, 92, 'MORTE'], [0.9264785377906193, 69, 'DUDA, A CARTEIRA']]
[[0.9352476091761823, 113, 'PATO COM LARANJA '], [0.94115101136635, 38, 'ENCOMENDA, A '], [0.944054576113554, 30, 'ACORDA']]
[[0.8927887465162205, 110, 'QUANDO AS VOZES CHEGAM'], [0.9245148643903602, 91, 'MORANGO'], [0.946962847721618, 82, 'QUERO SER JACK WHITE ']]


In [175]:
# Mídia - AKASHA
frase1 = "Este cientista maluco"
frase2 = "Você faz idéia do que isto pode representar para a evolução da humanidade?"
frase3 = "O céu azul, tem umas nuvens ali, uns pássaros"
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.9306624754718463, 33, 'AKASHA'], [0.947477426856111, 68, 'DONA CARMELA'], [0.9690210894595606, 41, 'JOB THAT SLOWLY KILLS YOU, A']]
[[0.8799038846461846, 33, 'AKASHA'], [0.9374235660714757, 43, 'BALA PERDIDA'], [0.937581221606764, 38, 'ENCOMENDA, A ']]
[[0.8702813695843072, 33, 'AKASHA'], [0.9269595040501025, 76, 'FLORES DA ESTRADA '], [0.9369006678271778, 116, 'ESCONDE-ESCONDE ']]


In [176]:
# Mídia - ANO EM QUE MEUS PAIS SAÍRAM DE FÉRIAS, O
frase1 = "Eu acho que exilado quer dizer quem tem um pai tão atrasado, mas tão atrasado que acaba nunca mais voltando pra casa."
frase2 = "o Brasil virou tricampeão mundial por 4 x 1 em cima da Itália"
frase3 = "Você não é judeu! Você é gói!"
distance_cosseno_bag(frase1, df)
distance_cosseno_bag(frase2, df)
distance_cosseno_bag(frase3, df)

[[0.8679010126831305, 91, 'MORANGO'], [0.8988233018514611, 41, 'JOB THAT SLOWLY KILLS YOU, A'], [0.9013986702816731, 83, 'JANELA SOBRE O SONHO']]
[[0.9643273989667829, 13, 'ANO EM QUE MEUS PAIS SAÍRAM DE FÉRIAS, O'], [0.9671357427701891, 51, 'CELEBRIDADE CAPÍTULO no.001'], [0.9690729866777145, 72, 'ENTRE A CRUZ E A CALDEIRA']]
[[0.9794044141907781, 13, 'ANO EM QUE MEUS PAIS SAÍRAM DE FÉRIAS, O'], [0.9919966699218872, 22, 'OLHOS AZUIS'], [1.0, 0, 'CAÇADOR DE DIAMANTES, O']]
